# Wirecard Data Challenge

## Case 1: - Modelagem dimensional

Um dos desafios é definir o modelo dimensional para a tarifação das contas na empresa. têm-se que efetuar consultas SQL em uma série de tabelas e considerando diversas regras para se conseguir obter a informação desejada. O primeiro desafio será definir um modelo dimensional que otimize o acesso a este tipo de informação.

Além das informações sobre as taxas, através do modelo dimensional proposto deve-se ser capaz de acessar algumas informações detalhadas das contas (login, nome, etc.).

#### Regras de Negócio

Obrigatoriamente, deve-se aplicar duas taxas em todo e qualquer pagamento: uma taxa fixa (definida em reais); e uma taxa percentual, que deve ser aplicada com base no valor total do pagamento. Os valores dessas taxas podem variar de acordo com o modelo de precificação.

Além dessas duas taxas, caso o cliente queira antecipar o valor que ele tem para receber, deve-se aplicar uma terceira taxa, a taxa de antecipação. Para esta taxa, as contas Moip são tarifadas de acordo com o modelo de precificação definido em contrato.

Atualmente temos três modelos de precificação:

    Modelo 1: Taxa percentual: aplicada de acordo com o tipo de pagamento.

Taxa de antecipação: aplicar uma taxa fixa de 1,99% a.m. (juros compostos)

    Modelo 2: Taxa percentual: aplicada de acordo com o tipo de pagamento e agrupamento de quantidade de parcelas.

Taxa de antecipação: aplicar uma taxa percentual de acordo com o meio de pagamento. Portanto, cada meio de pagamento terá sua taxa específica. Caso um meio de pagamento não tenha uma taxa atrelada, devemos considerar a mesma taxa de antecipação utilizada pelo meio de pagamento Cartão de crédito.

    Modelo 3:

Taxa percentual: aplicada de acordo com o tipo de pagamento e quantidade de parcelas.

Taxa de antecipação: aplicar uma taxa fixa de 2,89% a.m. (juros simples) sobre a quantidade de dias que serão antecipados.

Além dos 3 modelos descritos acima, embora cada uma das contas Moip possua suas respectivas taxas, caso ela transacione (receba pagamentos) através de um channel, as taxas que serão aplicadas àquela conta serão as taxas do channel, desde que a conta não possua uma taxa negociada.

Portanto, a precedência das taxas ocorre da seguinte forma (maior para menor prioridade): taxa da conta se houver negociação, taxa do channel e taxa da conta.

Um channel também possui uma conta Moip. Portanto, as taxas do channel são definidas da mesma forma que as taxas de uma conta Moip comum. Ou seja, seguindo as mesmas regras descritas acima, com exceção de que um channel não pode transacionar por outro channel.

## Modelo Dimensional

Dadas as características desejadas, será implementado banco de dados com arquitetura estrela ( ou estrela de Kimball), esse é o modelo dimensional aconselhado para essas situações no livro The Data Warehouse Toolkit, 3rd Edition.

O primeiro passo é avaliar qual processo será modelado em fatos. Na descrição do problema, logo no primeiro parágrafo pode-se identificar que o objetivo é modelar tarifação das contas na empresa.

O segundo passo seria definir qual a granulidade desses dados, mas como não vemos dimensão de tempo no banco de dados operacional, essa etapa não será feita.

Para identificar o que será fato e o que será dimensão, é melhor analisar o banco de dados operacional.

#### Análise do banco de dados operacional

Os dados de cada base de dados estão nos seguintes arquivos com formato CSV

In [1]:
from os import listdir
import os
filenames = listdir(os.getcwd())
filenames_list = [ filename for filename in filenames if filename.endswith( ".csv" ) ]
filenames_list

['account.csv',
 'accounts_channels.csv',
 'account_fixed_table_fee.csv',
 'account_payment_fee.csv',
 'channel.csv',
 'channel_fixed_table_fee.csv',
 'fixed_table_fee.csv',
 'member.csv',
 'payment_form.csv',
 'tax_applied_to_account.csv']

A partir da análise desses dados, pode-se construir o seguinte diagrama entidade-relação

<img src="wirecard_opdb.jpeg">

nota: pode ser que algumas dessas relações de cardinalidade estejam equivocadas

#### Fato: tarifação das contas na empresa

Como pode-se notar, existem 4 tabelas com informação sobre o valor de tarifação. Precisamos organizar os dados nessas tabelas de forma que essas informações sejam acessadas de forma mais conveniente por analistas e executivos a fim de se facilitar a tomada de decisão e planejamento.

Kimball recomenda em seu livro, preservar todos as informações relacionadas aos fatos possíveis, porque as necessidades do time de BI e analisatas pode mudar com o tempo, e dados irrelevantes hoje podem ser necessários para alguma análise no futuro, por isso, todas as informações nas tabelas serão preservadas na construção do modelo dimensional.

Outra preocupação que deve ser levada em conta, é utilizar nomes que façam sentido para todos os usuários, na hora de fazer o projeto de um Data Werehouse, é desencorajado o uso de abreaviações que não são convenção de todos ou que possam dificultar o entedimento de usuários no futuro.

Existem 3 tarifas principais sendo as duas primeiras obrigatórias,

1 - Tarifa Fixa
2 - Tarifa percentual, que deve ser aplicada com base no valor total do pagamento
3 - Tarifa de antecipação, caso o cliente queira antecipar o valor que ele tem para receber

A abordagem utilizada, será a de calcular o valor dessas tres tarifas utilizando os dados nas outras tabelas para deixar esse valores facilmente acessiveis na tabelas fato (a tabela central)

Com isso, uma possível implementação do esquema em estrela é como o abaixo:

<img src="wirecard_stardb.jpeg">


In [2]:
import pandas as pd
for filename in filenames_list:
    globals()[filename.strip('.csv')] = pd.read_csv(filename, sep=';')

In [3]:
account_payment_fee.head()

,id,account_id,payment_form_id,fixed,antecipation_percentage,one_parcel,between_two_and_six_parcels,more_than_seven_parcels
0,250358,1005890,1,0.00,0.00,3.99,0.00,0.00
1,250383,1005890,2,0.00,0.00,3.99,0.00,0.00
2,250367,1005890,3,0.00,0.00,3.99,3.99,3.99
3,250355,1005890,4,0.00,0.00,3.99,0.00,0.00
4,250378,1005890,5,0.00,0.00,3.99,3.99,3.99
